#  Segmenting and Clustering Project I.Guerrero

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

>Start by creating a new Notebook for this assignment.

>Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

>3. To create the above dataframe:
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
import requests
import lxml.html as lh


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
#let’s parse the first row as our header
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


In [5]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        


In [6]:
#Just to be sure, let’s check the length of each column.
[len(C) for (title,C) in col]

[181, 181, 181]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [9]:
#Replace al values of "\n" 
df = df.replace('\n','', regex=True)
df.rename(columns = {'Postal Code\n':'Postal Code','Borough\n':'Borough', 'Neighborhood\n': 'Neighborhood'}, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
#Drop the codes that are not yet assigned
df.replace('Not assigned',np.nan, inplace=True)
df.dropna(subset=['Borough'], axis=0, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
#Exploring the tast items of the DF
df.tail()

,Postal Code,Borough,Neighborhood
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing Centre
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
180,,Canadian postal codes,


In [12]:
#Reset the index and drop the last item since it is not useful
df.reset_index(inplace=True)
df.drop(['index'], axis=1 , inplace=True)

last_item = len(df) - 1
df.drop([last_item], inplace=True)

df.tail()

,Postal Code,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [13]:
print(df.shape)


(103, 3)


Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. \n

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html. \n

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this

In [14]:
'''
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


#address = {"query": {'postalCode': 'M8X', 'countryRegion': 'Canada', 'locality': 'Toronto', 'adminDistrict': 'Ontario'}}
address = "Canada, North York, Parkwoods"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
'''

'\nfrom geopy.geocoders import Nominatim # convert an address into latitude and longitude values\n\n\n#address = {"query": {\'postalCode\': \'M8X\', \'countryRegion\': \'Canada\', \'locality\': \'Toronto\', \'adminDistrict\': \'Ontario\'}}\naddress = "Canada, North York, Parkwoods"\ngeolocator = Nominatim(user_agent="toronto_explorer")\nlocation = geolocator.geocode(address)\nlatitude = location.latitude\nlongitude = location.longitude\nprint(\'The geograpical coordinate of Toronto are {}, {}.\'.format(latitude, longitude))\n'

In [15]:
'''
i = 0
column_names = ['Latitude', 'Longitude'] 

# instantiate the dataframe
lat_lng_coords = pd.DataFrame(columns=column_names)

for borough, neighborhood in zip(df['Borough'], df['Neighborhood']):
    first_neighborhood = neighborhood.split(",")
    neighborhood = first_neighborhood[0].replace("'", " ")

    address = borough +", "+ neighborhood

    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    i+=1
    print(i)
    lat_lng_coords = lat_lng_coords.append({'Latitude': latitude,
                                            'Longitude': longitude}, ignore_index=True)
    if i>20:
        break
 '''

'\ni = 0\ncolumn_names = [\'Latitude\', \'Longitude\'] \n\n# instantiate the dataframe\nlat_lng_coords = pd.DataFrame(columns=column_names)\n\nfor borough, neighborhood in zip(df[\'Borough\'], df[\'Neighborhood\']):\n    first_neighborhood = neighborhood.split(",")\n    neighborhood = first_neighborhood[0].replace("\'", " ")\n\n    address = borough +", "+ neighborhood\n\n    geolocator = Nominatim(user_agent="toronto_explorer")\n    location = geolocator.geocode(address)\n    latitude = location.latitude\n    longitude = location.longitude\n    i+=1\n    print(i)\n    lat_lng_coords = lat_lng_coords.append({\'Latitude\': latitude,\n                                            \'Longitude\': longitude}, ignore_index=True)\n    if i>20:\n        break\n '

In [16]:
#Import data set
df_latlon = pd.read_csv('http://cocl.us/Geospatial_data')
df_latlon.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
#Sort data frame of borough and Neighborhood
df_sorted = df.sort_values('Postal Code'). reset_index(drop=True)
#Sort data frame of Lat. and Lon.
df_latlon_sorted = df_latlon.sort_values('Postal Code').reset_index(drop=True)
#Join both frames and check in order is the same
df_complete = df_sorted.join(df_latlon_sorted, rsuffix = ' check order', sort = True)       
df_complete.head()


,Postal Code,Borough,Neighborhood,Postal Code check order,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [18]:
#Drop unwanted columns
df_complete = df_complete.drop('Postal Code check order', axis=1)
df_complete.head(12)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
